## 3.7 不可变映射类型

标准库里所有的映射类型都是可变的，但有时候你会有这样的需求，比如不能让用户错误地修改某个映射。  
从 Python 3.3 开始，types 模块中引入了一个封装类名叫 MappingProxyType。  
如果给这个类一个映射，它会返回一个只读的映射视图。虽然是个只读视图，但是它是动态的。这意味着如果对原映射做出了改动，我们通过这个视图可以观察到，但是无法通过这个视图对原映射做出修改。

#### 示例 3-9 用 MappingProxyType 来获取字典的只读实例 mappingproxy

In [3]:
from types import MappingProxyType

d = {1:'A'}  # 新建一个字典 d
d

{1: 'A'}

In [4]:
d_proxy = MappingProxyType(d)  # 建立 d 的一个视图 d_proxy
d_proxy

mappingproxy({1: 'A'})

In [5]:
d_proxy[1]  # 可以查看 d 的内容

'A'

In [6]:
d_proxy[2] = 'x'  # 通过 d_proxy 并不能做任何修改

TypeError: 'mappingproxy' object does not support item assignment

d_proxy 是动态的，也就是说对 d 所做的任何改动都会反馈到它上面

In [7]:
d[2] = 'B'
d_proxy

mappingproxy({1: 'A', 2: 'B'})

In [8]:
d_proxy[2]

'B'

## 3.8 集合论

集合的本质是许多唯一对象的聚集。因此，集合可以用于去重:

In [9]:
l = ['spam', 'spam', 'eggs', 'spam']
sl = set(l)
sl

{'eggs', 'spam'}

In [10]:
list(set(l))

['spam', 'eggs']

集合中的元素必须是可散列的，set 类型本身是不可散列的，但是 frozenset 可以。   
因此可以创建一个包含不同 frozenset 的 set。

In [13]:
hash(set([1,2]))

TypeError: unhashable type: 'set'

In [14]:
hash(frozenset([1,2]))

-1826646154956904602

In [16]:
{frozenset([1,2]),frozenset([2,3])}

{frozenset({2, 3}), frozenset({1, 2})}

除了保证唯一性，集合还实现了很多基础的中缀运算符。  
给定两个集合 a 和 b，a | b 返回的是它们的合集，a & b 得到的是交集，而 a - b 得到的是差集。  
合理地利用这些操作，不仅能够让代码的行数变少，还能减少 Python 程序的运行时间。  
这样做同时也是为了让代码更易读，从而更容易判断程序的正确性，因为利用这些运算符可以省去不必要的循环和逻辑操作。  

In [36]:
needles = {'123@qq.com','abc@qq.com','xyz@163.com','789@163.com','hhh@126.com'}  # 两种初始化方式
haystack = set(['kkk@qq.com','abc@qq.com','xyz@163.com','zzz@163.com'])

found = len(needles & haystack) # 交集
found

2

In [26]:
needles & haystack

{'abc@qq.com', 'xyz@163.com'}

In [27]:
needles | haystack

{'123@qq.com',
 '789@163.com',
 'abc@qq.com',
 'hhh@126.com',
 'kkk@qq.com',
 'xyz@163.com',
 'zzz@163.com'}

In [28]:
needles - haystack

{'123@qq.com', '789@163.com', 'hhh@126.com'}

In [29]:
# 另一种写法:
found = len(needles.intersection(haystack))
found

2

### 3.8.1 集合字面量

除空集之外，集合的字面量——{1}、{1, 2}，等等——看起来跟它的数学形式一模一样。  
如果是空集，那么必须写成 set() 的形式。

In [32]:
s = {1}
type(s)

set

In [33]:
s.pop()

1

In [35]:
s  # 空集

set()

像 {1, 2, 3} 这种字面量句法相比于构造方法(set([1, 2, 3]))要更快且更易读。  
后者的速度要慢一些，因为 Python 必须先从 set 这个名字来查询构造方法，然后新建一个列表，最后再把这个列表传入到构造方法里。  
但是如果是像 {1, 2, 3} 这样的字面量，Python 会利用一个专门的叫作 BUILD_SET 的字节码来创建集合。

In [37]:
from dis import dis

In [38]:
dis('{1}')

  1           0 LOAD_CONST               0 (1)
              2 BUILD_SET                1
              4 RETURN_VALUE


In [39]:
dis('set([1])')

  1           0 LOAD_NAME                0 (set)
              2 LOAD_CONST               0 (1)
              4 BUILD_LIST               1
              6 CALL_FUNCTION            1
              8 RETURN_VALUE


由于 Python 里没有针对 frozenset 的特殊字面量句法，我们只能采用构造方法。

In [40]:
frozenset(range(10))

frozenset({0, 1, 2, 3, 4, 5, 6, 7, 8, 9})

### 3.8.2 集合推导

#### 示例 3-13 新建一个 Latin-1 字符集合，该集合里的每个字符的 Unicode 名字里都 有“SIGN”这个单词

In [41]:
# 从 unicodedata 模块里导入 name 函数，用以获取字符的名字。
from unicodedata import name

# 把编码在 32~255 之间的字符的名字里有“SIGN”单词的挑出来，放到一个集合里。
{chr(i) for i in range(32, 256) if 'SIGN' in name(chr(i),'')}

{'#',
 '$',
 '%',
 '+',
 '<',
 '=',
 '>',
 '¢',
 '£',
 '¤',
 '¥',
 '§',
 '©',
 '¬',
 '®',
 '°',
 '±',
 'µ',
 '¶',
 '×',
 '÷'}

### 3.8.3 集合的操作

#### 集合的数学运算
交集

In [1]:
s = {'aaa', 'bbb', '12ab'}
z = {'123', '456', '12ab'}

s & z  # s 和 z 的交集

{'12ab'}

In [2]:
s.intersection(z)  # 等同于 s & z

{'12ab'}

In [6]:
s = {'aaa', 'bbb', '12ab'}
s &= z  # 把 s 更新为 s 和 z 的交集
s

{'12ab'}

In [7]:
s = {'aaa', 'bbb', '12ab'}
s.intersection_update(z)  # 把可迭代的 it 和其他所有参数转化为集合，然后求得它们与 s 的交集，然后把 s 更新成这个交集S
s

{'12ab'}

调用函数与使用运算符不同的地方在于，函数的参数并不一定是要是集合，而可以是所有可迭代类型，函数会自动将参数转化为集合

In [10]:
ls = ['123', '456', '12ab', '456']
s = {'aaa', 'bbb', '12ab'}
s.intersection_update(ls)  # 参数是列表，但是依然可以实现交集运算
s

{'12ab'}

并集

In [15]:
s = {'aaa', 'bbb', '12ab'}
z = {'123', '456', '12ab'}

s | z  # s 和 z 的交集

{'123', '12ab', '456', 'aaa', 'bbb'}

In [16]:
s.union(z)

{'123', '12ab', '456', 'aaa', 'bbb'}

In [17]:
s = {'aaa', 'bbb', '12ab'}
s |= z
s

{'123', '12ab', '456', 'aaa', 'bbb'}

In [18]:
s = {'aaa', 'bbb', '12ab'}
s.update(z)  # 注意，不是union_update
s

{'123', '12ab', '456', 'aaa', 'bbb'}

In [19]:
s = {'aaa', 'bbb', '12ab'}
ls = ['123', '456', '12ab']
s.update(ls)
s

{'123', '12ab', '456', 'aaa', 'bbb'}

差集

In [20]:
s = {'aaa', 'bbb', '12ab'}
z = {'123', '456', '12ab'}

s - z

{'aaa', 'bbb'}

In [21]:
s.difference(z)

{'aaa', 'bbb'}

In [22]:
s = {'aaa', 'bbb', '12ab'}
s -= z
s

{'aaa', 'bbb'}

In [24]:
s = {'aaa', 'bbb', '12ab'}
s.difference_update(z)
s

{'aaa', 'bbb'}

In [25]:
s = {'aaa', 'bbb', '12ab'}
ls = ['123', '456', '12ab']
s.difference_update(ls)
s

{'aaa', 'bbb'}

对称差集

In [26]:
s = {'aaa', 'bbb', '12ab'}
z = {'123', '456', '12ab'}

s ^ z

{'123', '456', 'aaa', 'bbb'}

In [27]:
s.symmetric_difference(z)

{'123', '456', 'aaa', 'bbb'}

In [28]:
s = {'aaa', 'bbb', '12ab'}
s ^= z
s

{'123', '456', 'aaa', 'bbb'}

In [29]:
s = {'aaa', 'bbb', '12ab'}
s.symmetric_difference_update(z)
s

{'123', '456', 'aaa', 'bbb'}

In [30]:
s = {'aaa', 'bbb', '12ab'}
ls = ['123', '456', '12ab']
s.symmetric_difference_update(ls)
s

{'123', '456', 'aaa', 'bbb'}